# Objectives:

1. Build a derived variable for tip as a percentage of the total fare.

2. Build a predictive model for tip as a percentage of the total fare. Use as much of the data as you like (or all of it). We will validate a sample.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import ttest_ind, f_oneway, lognorm, levy, skew, chisquare
from sklearn import preprocessing
from sklearn import preprocessing
from scipy.stats import skew
import numpy as np
from scipy.stats import boxcox

In [2]:
url = 'https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2015-09.csv'
df = pd.read_csv(url)

In [3]:
df.shape
df.columns

Index(['VendorID', 'lpep_pickup_datetime', 'Lpep_dropoff_datetime',
       'Store_and_fwd_flag', 'RateCodeID', 'Pickup_longitude',
       'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude',
       'Passenger_count', 'Trip_distance', 'Fare_amount', 'Extra', 'MTA_tax',
       'Tip_amount', 'Tolls_amount', 'Ehail_fee', 'improvement_surcharge',
       'Total_amount', 'Payment_type', 'Trip_type '],
      dtype='object')

### Objective 1

#### Lets Calculate tip as percentage of total fare...

Useful variables in consideration are...

Payment Type - Signifys how customer paid for the tip
    * 1 = Credit card 
    * 2 = Cash 
    * 3 = No charge 
    * 4 = Dispute 
    * 5 = Unknown 
    * 6 = Voided trip
    
Tip Amount - Populated via credit card and excludes cash tips
Total Amount - Amount charged to customers not including cash tips

Focusing on Credit Card Tip would be worthwile since others are not reported

In [4]:
# Tip as percetage of total fare
#Useful variables in computing tip percentage: Payment Type,Total Amount, Trip Amount
df['Tip_Percentage']= 100*(df.Tip_amount/df.Total_amount)
df.Tip_Percentage.describe()

count    1.490754e+06
mean     6.653826e+00
std      8.875158e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.666667e+01
max      1.000000e+02
Name: Tip_Percentage, dtype: float64

In [5]:
#Payment Type & Tip Amount - To determine if passengers that gave a tip
df.loc[:,'tip'] = df.Tip_amount.map(lambda x: 1 if x > 0 else 0)
df.groupby('Payment_type').tip.sum()

Payment_type
1    602729
2         2
3         0
4         1
5         0
Name: tip, dtype: int64

In [6]:
# As seen below, when payment type = 1
# Majority of the passengers paid the tip using a credit card as reported
# Also a majority of the passengers also paid using cash
# Few used other payment types
df.Payment_type.value_counts()

2    783699
1    701287
3      5498
4      4368
5        74
Name: Payment_type, dtype: int64

### Objective 2

As mentioned above, Tip Amount is only determined through credit card transcations and no cash tips are included. Therefore, It might be a wise decision to use only credit card payments made towards tips in building a predictive model

In [7]:
# Filter Payment_Type = Credit Card Payment
pt = df[df.Payment_type ==1]

In [8]:
pt.head(10)

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,improvement_surcharge,Total_amount,Payment_type,Trip_type,Tip_Percentage,tip
0,2,2015-09-01 00:02:34,2015-09-01 00:02:38,N,5,-73.979485,40.684956,-73.979431,40.685020,1,...,0.0,1.95,0.0,NaN,0.0,9.75,1,2.0,20.000000,1
1,2,2015-09-01 00:04:20,2015-09-01 00:04:24,N,5,-74.010796,40.912216,-74.010780,40.912212,1,...,0.0,0.00,0.0,NaN,0.0,45.00,1,2.0,0.000000,0
2,2,2015-09-01 00:01:50,2015-09-01 00:04:24,N,1,-73.921410,40.766708,-73.914413,40.764687,1,...,0.5,0.50,0.0,NaN,0.3,5.80,1,1.0,8.620690,1
5,2,2015-09-01 00:00:39,2015-09-01 00:05:20,N,1,-73.945297,40.808186,-73.937668,40.821198,1,...,0.5,1.36,0.0,NaN,0.3,8.16,1,1.0,16.666667,1
6,2,2015-09-01 00:00:52,2015-09-01 00:05:50,N,1,-73.890877,40.746426,-73.876923,40.756306,1,...,0.5,0.00,0.0,NaN,0.3,7.80,1,1.0,0.000000,0
8,2,2015-09-01 00:02:36,2015-09-01 00:07:20,N,1,-73.963150,40.693829,-73.956787,40.680531,1,...,0.5,1.46,0.0,NaN,0.3,8.76,1,1.0,16.666667,1
11,2,2015-09-01 00:04:00,2015-09-01 00:06:06,N,1,-73.905540,40.772526,-73.895363,40.768963,1,...,0.5,1.06,0.0,NaN,0.3,6.36,1,1.0,16.666667,1
17,2,2015-09-01 00:04:02,2015-09-01 00:07:20,N,1,-73.950989,40.680496,-73.953957,40.690422,1,...,0.5,1.26,0.0,NaN,0.3,7.56,1,1.0,16.666667,1
21,2,2015-09-01 00:04:22,2015-09-01 00:08:17,N,1,-73.912056,40.775402,-73.893646,40.768505,1,...,0.5,1.46,0.0,NaN,0.3,8.76,1,1.0,16.666667,1
22,2,2015-09-01 00:04:08,2015-09-01 00:10:00,N,1,-73.961746,40.713779,-73.985931,40.718048,6,...,0.5,1.86,0.0,NaN,0.3,11.16,1,1.0,16.666667,1


In [9]:
pt.dtypes

VendorID                   int64
lpep_pickup_datetime      object
Lpep_dropoff_datetime     object
Store_and_fwd_flag        object
RateCodeID                 int64
Pickup_longitude         float64
Pickup_latitude          float64
Dropoff_longitude        float64
Dropoff_latitude         float64
Passenger_count            int64
Trip_distance            float64
Fare_amount              float64
Extra                    float64
MTA_tax                  float64
Tip_amount               float64
Tolls_amount             float64
Ehail_fee                float64
improvement_surcharge    float64
Total_amount             float64
Payment_type               int64
Trip_type                float64
Tip_Percentage           float64
tip                        int64
dtype: object

In [10]:
# Determine Categorical Variables and Dummy Code accordingly
#df2.select_dtypes(include=['object'])
pt.loc[:,'lpep_pickup_datetime'] = pd.to_datetime(pt['lpep_pickup_datetime'])
pt.loc[:,'daymins'] = pt.lpep_pickup_datetime.map(lambda x: x.hour*60+x.minute)
pt.loc[:,'Store_and_fwd_flag'] = pt.Store_and_fwd_flag.map({'N':0,'Y':1})
pt.loc[:,'Trip_type '] = pt['Trip_type '].map({1:0,2:1})
pt.loc[:,'VendorID'] = pt.VendorID.map({1:0,2:1})
df_ratecodeid = pd.get_dummies(df['RateCodeID'])
pt = pd.concat([pt, df_ratecodeid], axis=1)

/Users/EshaSomavarapu/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/EshaSomavarapu/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [67]:
pt.dtypes

VendorID                        float64
lpep_pickup_datetime     datetime64[ns]
Lpep_dropoff_datetime            object
Store_and_fwd_flag              float64
RateCodeID                      float64
Pickup_longitude                float64
Pickup_latitude                 float64
Dropoff_longitude               float64
Dropoff_latitude                float64
Passenger_count                 float64
Trip_distance                   float64
Fare_amount                     float64
Extra                           float64
MTA_tax                         float64
Tip_amount                      float64
Tolls_amount                    float64
Ehail_fee                       float64
improvement_surcharge           float64
Total_amount                    float64
Payment_type                    float64
Trip_type                       float64
Tip_Percentage                  float64
tip                             float64
daymins                         float64
1                                 uint8


In [70]:
pt.isnull().sum()

VendorID                  793639
lpep_pickup_datetime      793639
Lpep_dropoff_datetime     793639
Store_and_fwd_flag        793639
RateCodeID                793639
Pickup_longitude          793639
Pickup_latitude           793639
Dropoff_longitude         793639
Dropoff_latitude          793639
Passenger_count           793639
Trip_distance             793639
Fare_amount               793639
Extra                     793639
MTA_tax                   793639
Tip_amount                793639
Tolls_amount              793639
Ehail_fee                1494926
improvement_surcharge     793639
Total_amount              793639
Payment_type              793639
Trip_type                 793642
Tip_Percentage            793959
tip                       793639
daymins                   793639
1                              0
2                              0
3                              0
4                              0
5                              0
6                              0
99        

In [71]:
# Due to time constraints of finishing this challenge and encoutering errors and not having enough time debug 
# those errors while building a model in predicting Tip Percentage, I have used SAS to make my predictions
# It is in a file called Question 4 _Objective 2. I will include the code I used to run in SAS as well as my sas file.